<a href="https://colab.research.google.com/github/alex-bolshunov/cst3602/blob/main/HW1_EDA_Oleksii_Bolshunov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Exploring Genetic Disease Data**

Your boss comes to you Monday morning and says “I figured out our next step; we are going to pivot from an online craft store and become a data center for genetic disease information! I found **ClinVar** which is a repository that contains expert curated data, and it is free for the taking. This is a gold mine! Look at the file and tell me what gene and mutation combinations are classified as dangerous.”

Make sure that you only give your boss the dangerous mutations and include:

1) Gene name

2) Mutation ID number

3) Mutation Position (chromosome & position)

4) Mutation value (reference & alternate bases)

5) Clinical significance (CLNSIG)

6) Disease that is implicated



This assignment will guide you through the construction of a deliverable for your boss.

Please use the **clinvar_final.txt** file for your analysis. You will need to download the file from Blackboard and then upload it into your notebook for use.

### File description

The file is VCF formatted.

VCF is a text file format which contains meta-information lines, a header line, and then data lines each containing information about a position in the genome. The format also can contain genotype information on samples for each position.

There are 8 fixed fields per record. All data lines are **tab-delimited**. In all cases, missing values are specified with a dot (‘.’).

1. CHROM - chromosome number
2. POS - position DNA nuceleotide count (bases) along the chromosome
3. ID - The unique identifier for each mutation
4. REF - reference base(s)
5. ALT - alternate base(s)
6. FILTER - filter status
7. QUAL - quality
8. INFO - a semicolon-separated series of keys with values in the format: <key>=<data>

Each row in the data represents a unique mutation.

### Sample Visual of ClinVar Data for Reference

```
##fileformat=VCFv4.1
##fileDate=2019-03-19
##source=ClinVar
##reference=GRCh38
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO
1	949523	rs786201005	C	T	.	.	GENEINFO=ISG15;CLNSIG=5
1	949696	rs672601345	C	CG	.	.	GENEINFO=ISG15;CLNSIG=5;CLNDN=Cancer
1	949739	rs672601312	G	T	.	.	GENEINFO=ISG15;CLNDBN=Cancer
1	955597	rs115173026	G	T	.	.	GENEINFO=AGRN;CLNSIG=2; CLNDN=Cancer
1	955619	rs201073369	G	C	.	.	GENEINFO=AGG;CLNDN=Heart_dis
1	957640	rs6657048	C	T	.	.	GENEINFO=AGG;CLNSIG=3;CLNDN=Heart_dis
1	976059	rs544749044	C	T	.	.	GENEINFO=AGG;CLNSIG=0;CLNDN=Heart_dis
```

In [264]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

**Please fill in all blanks and TODOs.**

# **Data Cleaning and Analysis**

**Load the Data and Rename Columns - 5pts**

In [265]:
#read in dataset using a tab delimiter and skipping all lines in the file that start with '#'
mutations = pd.read_csv('clinvar_final.txt', sep ='\t', comment='#')
mutations.head()

,CHROM,POS,ID,REF,ALT,FILTER,QUAL,INFO
0,1,1014O42,475283,G,A,.,.,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...
1,1,1O14122,542074,C,T,.,.,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...
2,1,1014143,183381,C,T,.,.,"ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:..."
3,1,1014179,542075,C,T,.,.,"ALLELEID=514896;CLNDISDB=MedGen:C4015293,OMIM:..."
4,1,1014217,475278,C,T,.,.,AF_ESP=0.00515;AF_EXAC=0.00831;AF_TGP=0.00339;...


In [266]:
#change column names to be more informative and to be all lowercase
mutations.columns = ['chromosome', 'position', 'mutation_id', 'reference_base', 'alternate_base', 'filter_status', 'quality', 'info']
mutations.head()

,chromosome,position,mutation_id,reference_base,alternate_base,filter_status,quality,info
0,1,1014O42,475283,G,A,.,.,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...
1,1,1O14122,542074,C,T,.,.,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...
2,1,1014143,183381,C,T,.,.,"ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:..."
3,1,1014179,542075,C,T,.,.,"ALLELEID=514896;CLNDISDB=MedGen:C4015293,OMIM:..."
4,1,1014217,475278,C,T,.,.,AF_ESP=0.00515;AF_EXAC=0.00831;AF_TGP=0.00339;...


In [267]:
#note the shape of the dataframe prior to any transformations
mutations.shape

(102321, 8)

**Examine Missing Values - 3pts**

In [268]:
#we know that all missing values in the dataset are specified with a dot ('.')
#print out all fields containing missing values
for column in list(mutations.columns):
    print(f'{column} missing values?', '.' in mutations[column].unique())

chromosome missing values? False
position missing values? False
mutation_id missing values? False
reference_base missing values? False
alternate_base missing values? True
filter_status missing values? True
quality missing values? True
info missing values? False


From the analysis above, we see that only three fields have missing values: *alternate_base*, *filter_status*, and *quality*.

Which of these columns are needed for our analysis / deliverable? Which are not?

**TODO - Written Answer**

There are three fields that have missing values: alternate_base, filter_status, and quality. For further analysis, we need alternate_base, because it's crucial for understanding genetic mutations and, therefore, diseases. We don't need filter_status and quality for further analysis, both of these columns contain missing values only.

**Drop Columns that are Not Needed - 2pts**

In [269]:
mutations = mutations.drop(columns = ['filter_status', 'quality'])
mutations.head()

,chromosome,position,mutation_id,reference_base,alternate_base,info
0,1,1014O42,475283,G,A,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...
1,1,1O14122,542074,C,T,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...
2,1,1014143,183381,C,T,"ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:..."
3,1,1014179,542075,C,T,"ALLELEID=514896;CLNDISDB=MedGen:C4015293,OMIM:..."
4,1,1014217,475278,C,T,AF_ESP=0.00515;AF_EXAC=0.00831;AF_TGP=0.00339;...


In [270]:
#look at dataframe shape
mutations.shape

(102321, 6)

**Data Anomaly in the Position Column - 5pts**

Some values in the position column have an 'O' instead of a 0. Assuming all position values should be numbers, we will replace all O's with 0's.

In [271]:
#replace O's with 0's in the position column
mutations['position'] = mutations.loc[:, 'position'].apply(lambda x: x.replace('O', '0'))
mutations.head()

,chromosome,position,mutation_id,reference_base,alternate_base,info
0,1,1014042,475283,G,A,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...
1,1,1014122,542074,C,T,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...
2,1,1014143,183381,C,T,"ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:..."
3,1,1014179,542075,C,T,"ALLELEID=514896;CLNDISDB=MedGen:C4015293,OMIM:..."
4,1,1014217,475278,C,T,AF_ESP=0.00515;AF_EXAC=0.00831;AF_TGP=0.00339;...


In [272]:
#convert position column to be an integer
mutations['position'] = pd.to_numeric(mutations['position'])
mutations['position'].dtype

dtype('int64')

**Examining the Info Field - 15 pts**

For our deliverable, we are interested in the following keys embedded within the info field:
- "GENEINFO": contains gene name
- "CLNSIG": contains clinical significance
- "CLNDN": contains disease name

Fill in the function below. As a hint, before writing the function, I recommend trying to manipulate the value in the "info" field for one record in the dataframe.

**Goal:** Transform the value from the source format to the target format.

Ex.

**Source Format:**

AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;ALLELEID=446939;CLNDISDB=MedGen:C4015293,OMIM:616126,Orphanet:ORPHA319563;CLNDN=Immunodeficiency_38_with_basal_ganglia_calcification;CLNHGVS=NC_000001.11:g.1014042G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=ISG15:9636;MC=SO:0001583|missense_variant;ORIGIN=1;RS=143888043'

**Target Format:**

{'CLNDN': 'Immunodeficiency_38_with_basal_ganglia_calcification',

 'CLNSIG': 'Benign',

 'GENEINFO': 'ISG15:9636'}






In [273]:
def extract_key_info(x):
  """This function takes in a string of information and returns a dictionary containing key-value pairs for the following keys:
        - GENEINFO
        - CLNSIG
        - CLNDN
  """
  pairs = x.split(';')
  keys = ('GENEINFO', 'CLNSIG', 'CLNDN')
  dict_holder = {}

  for pair in pairs:
      key, value = pair.split('=')
      if key in keys: dict_holder[key] = value

  return dict_holder

In [274]:
#create new column called "key_info" containing the dictionary returned by the extract_key_info function for each row
mutations['key_info'] = mutations['info'].apply(lambda x: extract_key_info(x))
mutations.head()

,chromosome,position,mutation_id,reference_base,alternate_base,info,key_info
0,1,1014042,475283,G,A,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...,{'CLNDN': 'Immunodeficiency_38_with_basal_gang...
1,1,1014122,542074,C,T,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...,{'CLNDN': 'Immunodeficiency_38_with_basal_gang...
2,1,1014143,183381,C,T,"ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:...",{'CLNDN': 'Immunodeficiency_38_with_basal_gang...
3,1,1014179,542075,C,T,"ALLELEID=514896;CLNDISDB=MedGen:C4015293,OMIM:...",{'CLNDN': 'Immunodeficiency_38_with_basal_gang...
4,1,1014217,475278,C,T,AF_ESP=0.00515;AF_EXAC=0.00831;AF_TGP=0.00339;...,{'CLNDN': 'Immunodeficiency_38_with_basal_gang...


**Add Columns for Gene Name, Clinical Significance, and Disease Name - 10pts**

In [275]:
#create three new columns: gene_name, clinical_significance, and disease_name using the key_info column
#impute a value of "Not_Given" if no value exists
#if GENEINFO, CLNSIG, or CLNDN are not present in the key_info field, then their value is not given
#hint: consider using a lambda function

mutations['gene_name'] = mutations['key_info'].apply(lambda x: x.get('GENEINFO', 'Not_Given'))
mutations['clinical_significance'] = mutations['key_info'].apply(lambda x: x.get('CLNSIG', 'Not_Given'))
mutations['disease_name'] = mutations['key_info'].apply(lambda x: x.get('CLNDN', 'Not_Given'))
mutations.head()

,chromosome,position,mutation_id,reference_base,alternate_base,info,key_info,gene_name,clinical_significance,disease_name
0,1,1014042,475283,G,A,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...,{'CLNDN': 'Immunodeficiency_38_with_basal_gang...,ISG15:9636,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...
1,1,1014122,542074,C,T,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...,{'CLNDN': 'Immunodeficiency_38_with_basal_gang...,ISG15:9636,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
2,1,1014143,183381,C,T,"ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:...",{'CLNDN': 'Immunodeficiency_38_with_basal_gang...,ISG15:9636,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
3,1,1014179,542075,C,T,"ALLELEID=514896;CLNDISDB=MedGen:C4015293,OMIM:...",{'CLNDN': 'Immunodeficiency_38_with_basal_gang...,ISG15:9636,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
4,1,1014217,475278,C,T,AF_ESP=0.00515;AF_EXAC=0.00831;AF_TGP=0.00339;...,{'CLNDN': 'Immunodeficiency_38_with_basal_gang...,ISG15:9636,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...


We should still have 102,321 rows. But now we have 10 columns. Let's check below.

In [276]:
mutations.shape

(102321, 10)

**Examine New Column Value Counts - 0pts**

In [277]:
#gene_name values
mutations['gene_name'].value_counts(dropna=False)

TTN:7273|TTN-AS1:100506866           6273
TTN:7273                             4775
Not_Given                            4718
MSH6:2956                            3442
MSH2:4436                            2806
                                     ... 
NCAPH:23397                             1
DCAF8:50717                             1
TSGA10:80705                            1
EIF4G3:8672                             1
STEAP3:55240|STEAP3-AS1:100874111       1
Name: gene_name, Length: 1477, dtype: int64

In [278]:
#clinical_significance values
mutations['clinical_significance'].value_counts(dropna=False)

Uncertain_significance                                                                      47980
Likely_benign                                                                               17885
Pathogenic                                                                                  12313
Likely_pathogenic                                                                            6269
Benign                                                                                       6138
Conflicting_interpretations_of_pathogenicity                                                 5404
Benign/Likely_benign                                                                         3338
Not_Given                                                                                    1797
Pathogenic/Likely_pathogenic                                                                  854
risk_factor                                                                                    98
association         

In [279]:
#disease_name values
mutations['disease_name'].value_counts(dropna=False)

not_specified                                                           14400
Not_Given                                                               12670
Hereditary_cancer-predisposing_syndrome                                  2951
Limb-girdle_muscular_dystrophy,_type_2J|Dilated_cardiomyopathy_1G        2588
Lynch_syndrome                                                           1717
                                                                        ...  
Neuroblastoma_3|Large_Cell/Anaplastic_Medulloblastoma                       1
Retinoblastoma|Neuroblastoma_3                                              1
Pilocytic_astrocytoma                                                       1
B_Lymphoblastic_Leukemia/Lymphoma_with_Hyperdiploidy|Neuroblastoma_3        1
Cowden_syndrome|Hereditary_cancer-predisposing_syndrome                     1
Name: disease_name, Length: 6140, dtype: int64

**Dataframe Filtering Pt. 1 - 5pts**

Our main interest is providing a list of harmful mutations to the boss. From the previous value counts analysis, we see that a mutation's harmfulness is captured in the clinical_significance field. So let's see how many records are not labeled.

In [280]:
len(mutations[mutations['clinical_significance'] == 'Not_Given']) / mutations.shape[0]

0.01756237722461665

Filter out all records where 'clinical_significance' is 'Not_Given'. Justify why we can do so.

**TODO - Written Answer**

We can filter out all records where "clinical_significance" is "Not_Given" because the goal of the analysis is to find mutations that possibly cause diseases. Therefore, the absence of clinical significance would not provide the required information to achieve this goal. Moreover, the number of missing values is relatively small and can be safely removed without significant impact on the analysis.

In [281]:
mutations_filtered = mutations[mutations['clinical_significance'] != 'Not_Given']
mutations_filtered.shape

(100524, 10)

**Defining which Mutations are Dangerous / Harmful - 0pts**

In [282]:
#look at unique values of clinical_significance field in the filtered mutations dataframe
mutations_filtered['clinical_significance'].unique()

array(['Benign', 'Uncertain_significance', 'Pathogenic', 'Likely_benign',
       'Benign/Likely_benign',
       'Conflicting_interpretations_of_pathogenicity',
       'Likely_pathogenic', 'Pathogenic/Likely_pathogenic', 'risk_factor',
       'Benign/Likely_benign,_risk_factor', 'association',
       'Pathogenic,_risk_factor', 'Benign/Likely_benign,_other',
       'Conflicting_interpretations_of_pathogenicity,_other,_risk_factor',
       'drug_response',
       'Conflicting_interpretations_of_pathogenicity,_risk_factor',
       'Affects', 'other',
       'Conflicting_interpretations_of_pathogenicity,_other',
       'Uncertain_significance,_risk_factor',
       'Pathogenic/Likely_pathogenic,_risk_factor',
       'Likely_pathogenic,_risk_factor',
       'Conflicting_interpretations_of_pathogenicity,_association',
       'protective', 'Pathogenic/Likely_pathogenic,_other',
       'Pathogenic,_other', 'Likely_benign,_other',
       'Pathogenic,_association,_protective', 'protective,_risk_fa

In [283]:
#reduce granularity of clinical significance by classifying the above unique values into
#the following three buckets: harmful, inconclusive, benign

harmful = np.array(['Pathogenic', 'Likely_pathogenic', 'Pathogenic/Likely_pathogenic', 'Pathogenic,_risk_factor',
           'Pathogenic/Likely_pathogenic,_risk_factor', 'Likely_pathogenic,_risk_factor', 'Pathogenic/Likely_pathogenic,_other',
           'Pathogenic,_other', 'Pathogenic,_association,_protective', 'Likely_pathogenic,_association', 'Pathogenic,_protective',
           'Pathogenic,_Affects', 'Likely_pathogenic,_other'])

benign = np.array(['Benign', 'Likely_benign', 'Benign/Likely_benign', 'Benign/Likely_benign,_risk_factor', 'Benign/Likely_benign,_other',
          'Likely_benign,_other', 'Benign,_other', 'Benign/Likely_benign,_protective', 'Benign,_drug_response',
          'Benign/Likely_benign,_association'])

inconclusive = np.array(['Uncertain_significance', 'Conflicting_interpretations_of_pathogenicity', 'risk_factor', 'association',
                'Conflicting_interpretations_of_pathogenicity,_other,_risk_factor', 'drug_response',
                'Conflicting_interpretations_of_pathogenicity,_risk_factor', 'Affects', 'other',
                'Conflicting_interpretations_of_pathogenicity,_other', 'Uncertain_significance,_risk_factor',
                'Conflicting_interpretations_of_pathogenicity,_association', 'protective', 'protective,_risk_factor',
                'Conflicting_interpretations_of_pathogenicity,_Affects',
                'Conflicting_interpretations_of_pathogenicity,_Affects,_association,_drug_response,_other',
                'Conflicting_interpretations_of_pathogenicity,_Affects,_other',
                'Conflicting_interpretations_of_pathogenicity,_protective', 'Affects,_risk_factor'])

To clarify the above buckets:

We are defining a mutation to be **dangerous/harmful** if its clinical_significance value contains some variation of "Pathogenic" or "Likely pathogenic".

We are defining a mutation to be **benign** if its clinical_significance value contains some variation of "Benign" or "Likely benign".

We are defining a mutation's harm to be **inconclusive** if its clinical_significance value is not a variation of "Pathogenic", "Likely pathogenic", "Benign", or "Likely benign".

**Dataframe Filtering Pt. 2 - 8pts**

Update the mutations_filtered dataframe to *exclude* all rows where the clinical significance is inconclusive or benign according to the definitions above.

In [284]:
#keep only harmful mutations and look at shape of resulting dataframe
mutations_filtered = mutations_filtered[mutations_filtered['clinical_significance'].apply(lambda x: x in harmful)]
mutations_filtered.shape

(19474, 10)

There are 19,474 harmful mutations in the file provided.

Can we filter any further?

The boss asked for gene and mutation combinations. Therefore, we can assume it is acceptable to remove records where gene_name is "Not_Given". Let's first check the number of records where this occurs.

In [285]:
#calculate proportion of harmful mutations where gene_name is "Not_Given"
len(mutations_filtered[mutations_filtered['gene_name'] == 'Not_Given']) / mutations_filtered.shape[0]

0.128016842970114

In [286]:
#number of harmful mutations where gene_name is "Not_Given"
len(mutations_filtered[mutations_filtered['gene_name'] == 'Not_Given'])

2493

Only **12.8%** of the harmful mutations are missing a gene name. Since the boss is looking explicitly for gene-mutation combinations and this quantity of mutations with missing gene names is so small, we will go ahead and filter these out.

In [287]:
#filter out mutations with a missing gene name and look at shape of resulting dataframe
mutations_filtered = mutations_filtered[mutations_filtered['gene_name'] != 'Not_Given']
mutations_filtered.shape

(16981, 10)

Let's return to our investigation of records missing alternate bases.

In [288]:
#calculate proportion of records with alternate_base missing
len(mutations_filtered[mutations_filtered['alternate_base'] == '.'])/mutations_filtered.shape[0]

0.00023555738766857076

In [289]:
#number of records with alternate_base missing
len(mutations_filtered[mutations_filtered['alternate_base'] == '.'])

4

The above proportion is so small that I assume it is safe to filter out mutations with a missing alternate base. Once we do this, all columns with the exception of disease_name will be fully populated, giving robust information on harmful gene-mutation combinations.

In [290]:
#filter out mutations with a missing alternate base and look at shape of resulting dataframe
mutations_filtered = mutations_filtered[mutations_filtered['alternate_base'] != '.']
mutations_filtered.shape

(16977, 10)

**Drop the *info* and *key_info* Columns - 0pts**

For the final dataframe, we can drop the "info" and "key_info" columns. We successfully extracted out all of the key information into the "gene_name", "clinical_significance", and "disease_name" columns.

In [291]:
mutations_filtered = mutations_filtered.drop(columns=['info', 'key_info'])
mutations_filtered.shape

(16977, 8)

**Examining Disease Name - 5pts**

How many disease names are 'Not_Given' or 'not_specified'?

In [292]:
unspecified_disease_names = len(mutations_filtered[(mutations_filtered['disease_name'] == 'not_specified') | (mutations_filtered['disease_name'] == 'Not_Given')])
unspecified_disease_names

3672

**TODO - Written Answer & Add supporting code cells.**

Discuss why these records should not be filterd out. How can we impute values for the missing disease names?

In [293]:
def get_index_prob(cdf:list):
  """
    accept cumulative distribution, return index based on the cdf
  """
  num = np.random.rand()
  return np.where(cdf >= num)[0][0]

In [294]:
#get pairs of names and counts
disease_name_pairs = list(mutations_filtered[(mutations_filtered['disease_name'] != 'not_specified') & (mutations_filtered['disease_name'] != 'Not_Given')]['disease_name'].value_counts().items())
total_specified = len(mutations_filtered) - unspecified_disease_names

#get names and relevant probability for each name
probabilities = [t[1] / total_specified for t in disease_name_pairs]
names = [t[0] for t in disease_name_pairs]

#get cumulitive distribution
cdf = np.cumsum(probabilities)

In [295]:
#first 10 values of disease names and relevant counts excluding Not_Given before imputation
mutations_filtered['disease_name'].value_counts()[1:11]

Lynch_syndrome                                     770
Hereditary_cancer-predisposing_syndrome            430
Ehlers-Danlos_syndrome,_type_4                     340
Primary_pulmonary_hypertension                     299
Nemaline_myopathy_2                                234
Inborn_genetic_diseases                            200
Severe_myoclonic_epilepsy_in_infancy               181
Hereditary_nonpolyposis_colon_cancer               177
Usher_syndrome,_type_2A|Retinitis_pigmentosa_39    176
Primary_dilated_cardiomyopathy                     165
Name: disease_name, dtype: int64

In [296]:
#randomly impute values using probabilities
mutations_filtered['disease_name'] = mutations_filtered['disease_name'].apply(lambda x: names[get_index_prob(cdf)] if x in ('Not_Given', 'not_specified') else x)

In [297]:
#first 10 values of disease names and relevant counts after imputation
mutations_filtered['disease_name'].value_counts()[0:10]

Lynch_syndrome                                     1008
Hereditary_cancer-predisposing_syndrome             542
Ehlers-Danlos_syndrome,_type_4                      420
Primary_pulmonary_hypertension                      405
Nemaline_myopathy_2                                 304
Inborn_genetic_diseases                             244
Severe_myoclonic_epilepsy_in_infancy                231
Hereditary_nonpolyposis_colon_cancer                225
Usher_syndrome,_type_2A|Retinitis_pigmentosa_39     223
Primary_dilated_cardiomyopathy                      216
Name: disease_name, dtype: int64

**TODO - Written Answer**

Filtering out the rows with disease name of "Not_Given" or "not_specified" can lead to inaccureate interpretations and biased results due to the loss of sugnifican portion of information. That's why the appropriate imputation technique is required to preserve the data. There are various technigques that can be applied in the analysis of categorical data. Some of them are constant value imputation, mode imputation, probabilistic imputaion, multiple imputaiton, prediction models, and KNN imputation. The probabilitstic imputation aims to generate imputed values that are represented of the underlying structure of a dataset. This is the technique that I used to impute missing values of "disease_name" in the dataset.

**Reorder Columns Intuitively and Create Final Dataframe for Boss - 2pts**

In [298]:
mutations_filtered = mutations_filtered[['gene_name', 'mutation_id', 'chromosome', 'position',
                    'reference_base', 'alternate_base', 'clinical_significance', 'disease_name']]

In [299]:
#create subset of harmful mutations for your boss - take the first 100 records
harmful_mutations_subset = mutations_filtered.iloc[:100]
harmful_mutations_subset

,gene_name,mutation_id,chromosome,position,reference_base,alternate_base,clinical_significance,disease_name
2,ISG15:9636,183381,1,1014143,C,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
8,ISG15:9636,161455,1,1014316,C,CG,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
9,ISG15:9636,161454,1,1014359,G,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
24,AGRN:375790,243036,1,1022225,G,A,Pathogenic,Congenital_myasthenic_syndrome
26,AGRN:375790,243037,1,1022313,A,T,Pathogenic,Congenital_myasthenic_syndrome
...,...,...,...,...,...,...,...,...
1143,PRDM16:63976,524100,1,3412184,GC,G,Likely_pathogenic,Roifman_syndrome
1152,PRDM16:63976,60724,1,3412301,A,T,Pathogenic,Left_ventricular_noncompaction_8
1292,CEP104:9731,221277,1,3816372,T,C,Pathogenic,Joubert_syndrome_25
1298,CEP104:9731,221275,1,3835081,G,GA,Pathogenic,Joubert_syndrome_25


# **Deliverable for Boss**

### **Final Mutations Dataframe and Discussion - 10pts**

In [300]:
harmful_mutations_subset

,gene_name,mutation_id,chromosome,position,reference_base,alternate_base,clinical_significance,disease_name
2,ISG15:9636,183381,1,1014143,C,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
8,ISG15:9636,161455,1,1014316,C,CG,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
9,ISG15:9636,161454,1,1014359,G,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
24,AGRN:375790,243036,1,1022225,G,A,Pathogenic,Congenital_myasthenic_syndrome
26,AGRN:375790,243037,1,1022313,A,T,Pathogenic,Congenital_myasthenic_syndrome
...,...,...,...,...,...,...,...,...
1143,PRDM16:63976,524100,1,3412184,GC,G,Likely_pathogenic,Roifman_syndrome
1152,PRDM16:63976,60724,1,3412301,A,T,Pathogenic,Left_ventricular_noncompaction_8
1292,CEP104:9731,221277,1,3816372,T,C,Pathogenic,Joubert_syndrome_25
1298,CEP104:9731,221275,1,3835081,G,GA,Pathogenic,Joubert_syndrome_25


**Sample Write Up Below**

Please fill in all blanks in the sample.

The ClinVar file provided contains a total of **19474** harmful mutations.

A harmful mutation is one whose clinical significance is equal to any of the following values:
- Pathogenic
- Likely_pathogenic
- Pathogenic/Likely_pathogenic
- Pathogenic,_risk_factor
- Pathogenic/Likely_pathogenic,_risk_factor
- Likely_pathogenic,_risk_factor
- Pathogenic/Likely_pathogenic,_other
- Pathogenic,_other
- Pathogenic,_association,_protective
- Likely_pathogenic,_association
- Pathogenic,_protective
- Pathogenic,_Affects
- Likely_pathogenic,_other

Out of the total number of harmful mutations, **2493 (12.8%)** were missing a gene name and **4 (0.02%)** were missing an alternate base. I excluded these mutations and focused on the remaining **16977** mutations.  

Of these remaining mutations, **3672** do not have a known disease that is implicated. However, I do *not* exclude these mutations. All features other than disease_name are fully populated for the 16,977 gene-mutation combinations. Therefore, in the next phase of analysis, I will impute these values **using the  probabilistic imputation technique. I will calculate the probabilities of each category occurring and then randomly sample from these probabilities to impute missing values. This technique will ensure the imputed values maintain the distribution of observed values as closely as possible and preserve the characteristics of the original dataset.**

The dataframe displayed above contains the first 100 harmful mutations from the set of 16,977.  Each row represents a unique mutation.

**A Note on Clinical Significance Values**

The original ClinVar file contained **102321** mutations. Of these, approximately **1.8%** were missing a clinical significance value.  Therefore, the harmfulness of these mutations could not be determined. On account of this fact and the small percentage of records affected, I removed these records. There were **100524** mutations remaining.

After categorizing the various clinical significance values as "harmful", "benign", or "inconclusive", I filtered out all "benign" and "inconclusive" mutations.